In [ ]:
%load_ext autoreload
%autoreload 2

import os

from toy_fish_data_uploader import rectify_and_upload_images, process_into_plali_records, \
    establish_plali_connection, insert_into_plali, get_capture_keys, get_image_pair_from_capture_key


In [ ]:
os.environ['PLALI_SQL_CREDENTIALS'] = '/run/secrets/plali_sql_credentials'

In [ ]:
s3_bucket = 'aquabyte-images-raw'
s3_prefix = 'environment=production/site-id=55/pen-id=97/date=2021-03-17/hour=10'
stereo_parameters_url = 'https://aquabyte-stereo-parameters.s3-eu-west-1.amazonaws.com/L40029797_R40020184/2021-02-25T11:30:42.149694000Z_L40029797_R40020184_stereo-parameters.json'
metadata = { 'type': 'Dale P3 pre-swap -- Round 2' }
workflow_id = '00000000-0000-0000-0000-000000000055'

capture_keys = get_capture_keys(s3_bucket, s3_prefix)
image_pairs = [get_image_pair_from_capture_key(capture_key) for capture_key in capture_keys]
image_url_pairs = rectify_and_upload_images(s3_bucket, s3_prefix, stereo_parameters_url, image_pairs)


In [ ]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [ ]:
values_to_insert = process_into_plali_records(image_url_pairs, metadata, workflow_id)
engine, sql_metadata = establish_plali_connection()

n = 10
count = 0
for chunk in chunker(values_to_insert[:], n):
    insert_into_plali(chunk, engine, sql_metadata)
    
    count += 1
    print(count)
    